In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder

import os
import requests


import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from google.protobuf.json_format import MessageToDict

from ml_metadata.proto import metadata_store_pb2

import pprint
pp = pprint.PrettyPrinter()

## 2. Descripcíón del dataset

### 2.1 Cargar el Dataset

In [3]:
## download the dataset
# Directory of the raw data files
_data_root = '../data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [4]:
df_full = pd.read_csv(_data_filepath, dtype={'Soil_Type': object})
df_full.shape

(116203, 13)

In [5]:
df_full.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [6]:
df_full['Cover_Type'].value_counts()

1    56720
0    42307
2     7228
6     4045
5     3478
4     1892
3      533
Name: Cover_Type, dtype: int64

In [7]:
df_full['Soil_Type'].value_counts()

C7745    22912
C7202    11560
C7756    10539
C7757     9178
C7201     6690
C4703     6541
C7746     6068
C4744     5961
C7755     5130
C7700     4244
C4758     3443
C8771     3113
C8772     2720
C2705     2488
C4704     2396
C7102     1881
C8776     1729
C2703     1478
C2717     1378
C2704      977
C7101      787
C6102      690
C2702      592
C6101      565
C7702      539
C8703      385
C6731      375
C7790      324
C2706      322
C4201      257
C7709      242
C7710      194
C7103      156
C5101      123
C7701      101
C8708       63
C3502       24
C3501       19
C8707       18
C5151        1
Name: Soil_Type, dtype: int64

## 3. Seleccion de Caracteristicas

In [8]:
# Extraer solo las columnas numéricas
numerical_features = df_full.select_dtypes(include=['number'])

# Definir la variable objetivo (categórica)
y = df_full['Cover_Type'] 

# Preparar características de entrada
X = numerical_features.drop(columns=['Cover_Type'], errors='ignore')

# Aplicar SelectKBest con f_classif
selector = SelectKBest(score_func=f_classif, k='all')  # 'all' para evaluar todas las características
X_new = selector.fit_transform(X, y)

# Crear un DataFrame con los puntajes
scores = pd.DataFrame({'Feature': X.columns, 'Score': selector.scores_})

# Determinar qué características retener según un umbral 
threshold = scores['Score'].quantile(0.3) 
scores['Retain'] = scores['Score'] >= threshold

# Mostrar las características seleccionadas
print(scores.sort_values(by='Score', ascending=False))

                              Feature         Score  Retain
0                           Elevation  31087.079657    True
5     Horizontal_Distance_To_Roadways   2050.314700    True
2                               Slope   1559.369722    True
9  Horizontal_Distance_To_Fire_Points   1452.737911    True
6                       Hillshade_9am    643.862634    True
7                      Hillshade_Noon    595.275348    True
3    Horizontal_Distance_To_Hydrology    488.221292    True
4      Vertical_Distance_To_Hydrology    241.029561   False
8                       Hillshade_3pm    190.497164   False
1                              Aspect     88.987497   False


In [9]:
cat_features = df_full.select_dtypes(exclude=['number'])
cat_features

,Wilderness_Area,Soil_Type
0,Commanche,C7202
1,Commanche,C7757
2,Rawah,C7745
3,Rawah,C7745
4,Rawah,C4744
...,...,...
116198,Commanche,C7756
116199,Rawah,C7745
116200,Rawah,C7202
116201,Rawah,C7745


In [10]:
selected_features = scores[scores['Retain'] == True]['Feature'].tolist() + cat_features.columns.tolist()

In [11]:
selected_features

['Elevation',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area',
 'Soil_Type']

In [12]:
df_full_selected_features = df_full[selected_features + ['Cover_Type']]
df_full_selected_features.shape

(116203, 10)

In [13]:
df_full_selected_features

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,3596,231,242,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,2275,200,253,866,Commanche,C7756,1
116199,3125,13,234,2430,224,212,1426,Rawah,C7745,0
116200,3166,11,67,1275,234,240,2404,Rawah,C7202,0
116201,3154,14,738,6012,181,239,1320,Rawah,C7745,1


## 4. Data Pipeline

### 4.1 Configurar el contexto interactivo

In [14]:
_pipeline_root = './pipeline/'
# Crear el contexto interactivo
context = InteractiveContext(pipeline_root=_pipeline_root)

### 4.2 Generando Ejemplos

In [15]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=_data_root)

In [16]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [17]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/7


In [18]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [19]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [20]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [21]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'Aspect': {'int64List': {'value': ['3']}},
                           'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_3pm': {'int64List': {'value': ['144']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},
                           'Hillshade_Noon': {'int64List': {'value': ['202']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1557']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['485']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['2495']}},
                           'Slope': {'int64List': {'value': ['18']}},
                           'Soil_Type': {'bytesList': {'value': ['Qzc3NTc=']}},
                           'Vertical_Distance_To_Hydrology': {'int64List': {'value': ['71']}},


2025-03-02 17:45:08.154068: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### 4.3 Estadísticas

In [22]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [23]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### 4.4 Inferir el esquema

In [24]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [25]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### 4.5 Curando el esquema

In [26]:
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

In [27]:
# Obtener la ruta del directorio donde se guardó el esquema original
schema_dir = schema_gen.outputs['schema'].get()[0].uri

# Construir la ruta del archivo de esquema (schema.pbtxt)
schema_path = os.path.join(schema_dir, "schema.pbtxt")

# Cargar el esquema
schema = tfdv.load_schema_text(schema_path)

# Definir dominios de valores permitidos
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(min=0, max=90))
tfdv.set_domain(schema, "Cover_Type", schema_pb2.IntDomain(min=0, max=6, is_categorical=True))  

# Guardar el esquema actualizado en un nuevo archivo
updated_schema_path = os.path.join(schema_dir, "updated_schema.pbtxt")
tfdv.write_schema_text(schema, updated_schema_path)

### 4.6 Entornos de esquema

In [28]:
# Obtener la ruta del esquema generado
schema_dir = schema_gen.outputs['schema'].get()[0].uri
schema_path = os.path.join(schema_dir, "updated_schema.pbtxt")

# Cargar el esquema inferido
schema = tfdv.load_schema_text(schema_path)

# 1. Simular el conjunto de datos de servicio
df_train = pd.read_csv(_data_filepath)  # Reemplazar con el dataset real
df_serving = df_train.drop(columns=["Cover_Type"])  # Eliminar la etiqueta

# Generar estadísticas para el conjunto de inferencia
serving_stats = tfdv.generate_statistics_from_dataframe(df_serving)

# 2. Configurar los entornos en el esquema
# Todas las características están en ambos entornos por defecto
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# Especificar que 'Cover Type' no está en SERVING
tfdv.get_feature(schema, "Cover_Type").not_in_environment.append("SERVING")

# 3. Validar los datos de servicio con el esquema y entornos configurados
serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment="SERVING")

# Mostrar anomalías si existen
tfdv.display_anomalies(serving_anomalies_with_env)

# 4. Guardar el esquema actualizado en un archivo
updated_schema_path = os.path.join(schema_dir, "final_schema.pbtxt")
tfdv.write_schema_text(schema, updated_schema_path)

# Verificar que los cambios se guardaron correctamente
schema_updated = tfdv.load_schema_text(updated_schema_path)
print(schema_updated)

feature {
  name: "Aspect"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Cover_Type"
  type: INT
  int_domain {
    min: 0
    max: 6
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVING"
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Elevation"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_3pm"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_9am"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_Noon"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_

### 4.7 Genere nuevas estadísticas usando el esquema actualizado

In [29]:
from tfx.v1.components import ImportSchemaGen, StatisticsGen

# 1. Importar el esquema curado a TFX como un artefacto
import_schema_gen = ImportSchemaGen(schema_file=updated_schema_path)
context.run(import_schema_gen)

# 2. Crear StatisticsGen usando el esquema curado
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"],
    schema=import_schema_gen.outputs["schema"]  # Pasar el esquema actualizado
)

# 3. Ejecutar StatisticsGen
context.run(statistics_gen)

# 4. Visualizar las estadísticas generadas con el nuevo esquema
context.show(statistics_gen.outputs["statistics"])


### 4.8 Comprobar anomalías

In [30]:
from tfx.v1.components import ExampleValidator

# 1. Crear el componente ExampleValidator con estadísticas y esquema actualizados
validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=import_schema_gen.outputs["schema"]  # Usar el esquema actualizado
)

# 2. Ejecutar ExampleValidator
context.run(validate_stats)

# 3. Visualizar las anomalías detectadas (si las hay)
context.show(validate_stats.outputs["anomalies"])


### 4.9 Ingeniería de características

In [35]:
# Set the transform module filename
_constants_module_file = 'constants_preprocessing.py'

In [36]:
%%writefile {_constants_module_file}

# Definir nombres de características y tipos
NUMERIC_FEATURE_KEYS = [
    'Elevation',
     'Slope',
     'Horizontal_Distance_To_Hydrology',
     'Horizontal_Distance_To_Roadways',
     'Hillshade_9am',
     'Hillshade_Noon',
     'Horizontal_Distance_To_Fire_Points'
]
CATEGORICAL_FEATURE_KEYS = ["Wilderness_Area", 'Soil_Type']
LABEL_KEY = "Cover_Type"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing constants_preprocessing.py


### 4.10 Función de preprocesamiento

In [ ]:
# Set the transform module filename
_module_file = 'preprocessing.py'

In [39]:
%%writefile {_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import constants_preprocessing

NUMERIC_FEATURE_KEYS = constants_preprocessing.NUMERIC_FEATURE_KEYS
CATEGORICAL_FEATURE_KEYS = constants_preprocessing.CATEGORICAL_FEATURE_KEYS
LABEL_KEY = constants_preprocessing.LABEL_KEY
transformed_name = constants_preprocessing.transformed_name

# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Bucketize these features
    #for key in BUCKET_FEATURE_KEYS:
     #   outputs[transformed_name(key)] = tft.bucketize(
      #      inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in CATEGORICAL_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[transformed_name(LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[LABEL_KEY])

    return outputs

Overwriting preprocessing.py


### 4.11 Transformar

In [40]:
from tfx.v1.components import Transform

# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=import_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_module_file))

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build/lib
copying constants_preprocessing.py -> build/lib
copying preprocessing.py -> build/lib
installing to /tmp/tmpwxtscflg
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpwxtscflg/.
copying build/lib/constants_preprocessing.py -> /tmp/tmpwxtscflg/.
running install_egg_info
running egg_info


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpwxtscflg/./tfx_user_code_Transform-0.0+ef1161f44cb59dd880437f32e3956c6e499fe1c83e40b6ffdff8f4eb1f5c5b39-py3.10.egg-info
running install_scripts
creating /tmp/tmpwxtscflg/tfx_user_code_Transform-0.0+ef1161f44cb59dd880437f32e3956c6e499fe1c83e40b6ffdff8f4eb1f5c5b39.dist-info/WHEEL
creating '/tmp/tmpt6h_3d84/tfx_user_code_Transform-0.0+ef1161f44cb59dd880437f32e3956c6e499fe1c83e40b6ffdff8f4eb1f5c5b39-py3-none-any.whl' and adding '/tmp/tmpwxtscflg' to it
adding 'constants_preprocessing.py'
addi

Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+ef1161f44cb59dd880437f32e3956c6e499fe1c83e40b6ffdff8f4eb1f5c5b39-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+ef1161f44cb59dd880437f32e3956c6e499fe1c83e40b6ffdff8f4eb1f5c5b39-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 14
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [41]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

In [42]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [43]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.5072901]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.7559055]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.79527557]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.2172154]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.3540146]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.35485706]}},
                           'Slope_xf': {'floatList': {'value': [0.27272728]}},
                           'Soil_Type_xf': {'int64List': {'value': ['3']}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['1']}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['1']}},
                           'Eleva

2025-03-02 18:01:23.499339: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 5. Metadatos de aprendizaje automático

### 5.1 Acceso a artefactos seleccionados

In [ ]:
# Usar el metadata_store del InteractiveContext
store = context.metadata_store

# Listar y mostrar todos los tipos de artefactos registrados
print("=== Tipos de Artefactos en el Almacén de ML Metadata ===")
artifact_types = store.get_artifact_types()
for at in artifact_types:
    print(f"Tipo: {at.name}")

# Obtener todos los artefactos del tipo "Schema"
print("\n=== Artefactos del tipo 'Schema' ===")
schema_artifacts = store.get_artifacts_by_type("Schema")

# Mostrar al menos dos filas (ejemplo: esquema inferido y actualizado)
print("{:<12} {:<15} {}".format("Artifact id", "Type", "URI"))
for art in schema_artifacts:
    print("{:<12} {:<15} {}".format(art.id, art.type.name, art.uri))

### 5.2 Seguimiento de artefactos

In [ ]:
def get_parent_artifacts(store, artifact_id):
    """
    Dado el ID de un artefacto, retorna una lista de los artefactos principales (entradas) 
    que se usaron en la ejecución que produjo ese artefacto.
    """
    parent_artifacts = {}
    events = store.get_events_by_artifact_ids([artifact_id])

    for event in events:
        execution_id = event.execution_id
        events_for_execution = store.get_events_by_execution_ids([execution_id])
        for ev in events_for_execution:
            if ev.type == metadata_store_pb2.Event.INPUT:
                if ev.artifact_id not in parent_artifacts:
                    artifact_list = store.get_artifacts_by_id([ev.artifact_id])
                    if artifact_list:
                        parent_artifacts[ev.artifact_id] = artifact_list[0]
                        
    # Retornar la lista de artefactos principales
    return list(parent_artifacts.values())

### 5.3 Obtener artefactos principales

In [ ]:
# Obtener el artefacto TransformGraph
transform_graph_artifact = transform.outputs['transform_graph'].get()[0]
print("\n=== Información del Artefacto TransformGraph ===")
print(f"Artifact id: {transform_graph_artifact.id}")
print(f"URI: {transform_graph_artifact.uri}")

# Rastrear y obtener los artefactos principales asociados
parent_arts = get_parent_artifacts(store, transform_graph_artifact.id)

print("\n=== Artefactos principales asociados a TransformGraph ===")
print("{:<12} {:<15} {}".format("Artifact id", "Type", "URI"))
for art in parent_arts:
    print("{:<12} {:<15} {}".format(art.id, art.type.name, art.uri))